<a href="https://colab.research.google.com/github/AndrewBoessen/CSCI3387_Notebooks/blob/main/Gym_Super_Mario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym
!pip install gym-super-mario-bros

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.8 MB/s eta 0:00:00
  Created wheel for nes-py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=535719 sha256=d309c1612dc379aef8d996118a5a43cc15ef73c9526d901dbb296426dcac9d4f
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes-py


In [ ]:
import gym
import numpy as np
import torch
from gym.wrappers import GrayScaleObservation, ResizeObservation
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import os

## Environment Setup

env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = GrayScaleObservation(env, keep_dim=False)
env = ResizeObservation(env, (84, 84))

## Random Policy

class RandomPolicy:
    def __init__(self, action_space):
        self.action_space = action_space

    def action(self, observation):
        return self.action_space.sample()

policy = RandomPolicy(env.action_space)

## Data Collection

def collect_episode(env, policy):
    observations = []
    actions = []
    observation = env.reset()
    done = False
    while not done:
        action = policy.action(observation)
        observations.append(observation)
        actions.append(action)
        observation, _, done, _ = env.step(action)
    return observations, actions

## Dataset Creation

def create_datasets(num_episodes, eval_prob=0.1):
    all_observations = []
    all_actions = []
    eval_observations = []
    eval_actions = []

    for episode in range(num_episodes):
        observations, actions = collect_episode(env, policy)

        if np.random.random() < eval_prob:
            eval_observations.extend(observations)
            eval_actions.extend(actions)
        else:
            all_observations.extend(observations)
            all_actions.extend(actions)

    return (torch.tensor(np.array(all_observations)),
            torch.tensor(np.array(all_actions)),
            torch.tensor(np.array(eval_observations)),
            torch.tensor(np.array(eval_actions)))

## Data Sharding

def shard_data(data, shard_size=1000):
    return [data[i:i+shard_size] for i in range(0, len(data), shard_size)]

## Save Data

def save_shards(shards, prefix, directory):
    os.makedirs(directory, exist_ok=True)
    for i, shard in enumerate(shards):
        torch.save(shard, os.path.join(directory, f"{prefix}_{i}.pt"))

## Main Execution

num_episodes = 10
train_obs, train_actions, eval_obs, eval_actions = create_datasets(num_episodes)

train_obs_shards = shard_data(train_obs)
train_actions_shards = shard_data(train_actions)
eval_obs_shards = shard_data(eval_obs)
eval_actions_shards = shard_data(eval_actions)

save_shards(train_obs_shards, "train_obs", "data/train")
save_shards(train_actions_shards, "train_actions", "data/train")
save_shards(eval_obs_shards, "eval_obs", "data/eval")
save_shards(eval_actions_shards, "eval_actions", "data/eval")

print(f"Saved {len(train_obs)} training samples and {len(eval_obs)} evaluation samples.")

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is reco

RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 448 vs 342